
# Iteration 2 — 1.3.8 Data Analysis on AB Dataset
**Team 102D · AB Data Challenge**

This notebook implements Task **1.3.8** in three sub‑steps:
- **1.3.8.1 Data Quality Re‑check**
- **1.3.8.2 Exploratory Analysis on New/Updated Variables**
- **1.3.8.3 Adjustment of Cleaning Rules if Needed**

> Notes
> - We start by converting **Parquet → CSV** to quickly inspect data (requested by the team).
> - Keep the code **simple and well‑commented**. We only adjust cleaning if necessary.
> - Outputs are written to `results/iteration_2/` for traceability.


In [1]:

# === Setup & paths ===
import os
import glob
import json
import pandas as pd

# Root of your repo (adjust if running directly inside the project)
PROJECT_ROOT = r"C:\Users\joan\Desktop\FEINA\UPF\Course\Fourth_year\Primer_Trimestre\Project_Management\AB\AB_DataChallenge.Team102D"  # set to your repo root when moving this notebook
DATA_DIR = os.path.join(PROJECT_ROOT, "data")  # expects parquet files here
RESULTS_DIR = os.path.join(PROJECT_ROOT, "results", "iteration_2")
DERIVED_DIR = os.path.join(DATA_DIR, "derived")
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(DERIVED_DIR, exist_ok=True)

# Helper: safe write
def _save(df, path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    df.to_csv(path, index=False)
    print(f"[saved] {path}")



## Step 0 — Quick Conversion: Parquet → CSV
We convert available Parquet files under `data/` to CSV for immediate inspection.


In [2]:

# Find Parquet files inside data/
parquet_files = glob.glob(os.path.join(DATA_DIR, "*.parquet"))
print("Found Parquet files:", parquet_files)

converted_csv_paths = []
for pq in parquet_files:
    try:
        df_tmp = pd.read_parquet(pq)
    except Exception as e:
        print(f"[warn] Could not read {pq}: {e}")
        continue
    base = os.path.splitext(os.path.basename(pq))[0]
    out_csv = os.path.join(DERIVED_DIR, f"{base}.csv")
    df_tmp.to_csv(out_csv, index=False)
    converted_csv_paths.append(out_csv)
    print(f"[ok] Parquet→CSV: {pq} -> {out_csv}")

# Pick the first CSV as our working file (adjust if you have multiple tables)
if converted_csv_paths:
    WORK_CSV = converted_csv_paths[0]
    print("Using WORK_CSV:", WORK_CSV)
else:
    # If you have multiple input tables (telelectura, anomalies, billing, etc.), 
    # add them below once available (placeholders for now).
    WORK_CSV = None
    print("[info] No CSV produced yet. Place your parquet files in `data/`.")


Found Parquet files: ['C:\\Users\\joan\\Desktop\\FEINA\\UPF\\Course\\Fourth_year\\Primer_Trimestre\\Project_Management\\AB\\AB_DataChallenge.Team102D\\data\\big_dataset.parquet', 'C:\\Users\\joan\\Desktop\\FEINA\\UPF\\Course\\Fourth_year\\Primer_Trimestre\\Project_Management\\AB\\AB_DataChallenge.Team102D\\data\\dataset_sample.parquet']
[ok] Parquet→CSV: C:\Users\joan\Desktop\FEINA\UPF\Course\Fourth_year\Primer_Trimestre\Project_Management\AB\AB_DataChallenge.Team102D\data\big_dataset.parquet -> C:\Users\joan\Desktop\FEINA\UPF\Course\Fourth_year\Primer_Trimestre\Project_Management\AB\AB_DataChallenge.Team102D\data\derived\big_dataset.csv
[ok] Parquet→CSV: C:\Users\joan\Desktop\FEINA\UPF\Course\Fourth_year\Primer_Trimestre\Project_Management\AB\AB_DataChallenge.Team102D\data\dataset_sample.parquet -> C:\Users\joan\Desktop\FEINA\UPF\Course\Fourth_year\Primer_Trimestre\Project_Management\AB\AB_DataChallenge.Team102D\data\derived\dataset_sample.csv
Using WORK_CSV: C:\Users\joan\Desktop\FEI

In [3]:
# === Compare dataset_sample.csv vs big_dataset.csv ===
import pandas as pd
import os

# Paths (adapt to your structure)
sample_path = os.path.join(DERIVED_DIR, "dataset_sample.csv")
big_path = os.path.join(DERIVED_DIR, "big_dataset.csv")

# Load both (use low_memory=False to avoid dtype warnings)
df_sample = pd.read_csv(sample_path, low_memory=False)
df_big = pd.read_csv(big_path, low_memory=False)

print("✅ Files loaded")
print(f"Sample shape: {df_sample.shape}")
print(f"Big dataset shape: {df_big.shape}\n")

# --- 1️⃣ Column comparison ---
sample_cols = set(df_sample.columns)
big_cols = set(df_big.columns)
print("Columns only in sample:", sample_cols - big_cols)
print("Columns only in big:", big_cols - sample_cols)

# --- 2️⃣ Data type comparison (for shared columns) ---
shared_cols = list(sample_cols & big_cols)
dtype_diff = {
    c: (df_sample[c].dtype, df_big[c].dtype)
    for c in shared_cols
    if df_sample[c].dtype != df_big[c].dtype
}
print("\nColumns with different dtypes:", dtype_diff)

# --- 3️⃣ Basic row-level diagnostics ---
print("\nHead of sample:")
display(df_sample.head(3))
print("\nHead of big:")
display(df_big.head(3))

# --- 4️⃣ Basic numeric summary comparison ---
num_cols = [c for c in shared_cols if pd.api.types.is_numeric_dtype(df_big[c])]
summary_sample = df_sample[num_cols].describe().T
summary_big = df_big[num_cols].describe().T
diff_summary = summary_sample.join(summary_big, lsuffix="_sample", rsuffix="_big", how="outer")
display(diff_summary)

# --- 5️⃣ Optional: check overlap if unique identifier exists ---
id_candidates = [c for c in df_big.columns if "POLIZA" in c.upper() or "ID" in c.upper()]
if id_candidates:
    id_col = id_candidates[0]
    overlap = len(set(df_sample[id_col]) & set(df_big[id_col]))
    print(f"\n{overlap} IDs overlap out of {df_sample[id_col].nunique()} in sample and {df_big[id_col].nunique()} in big.")
else:
    print("\nNo clear ID column found for overlap check.")


✅ Files loaded
Sample shape: (25977, 7)
Big dataset shape: (25977, 7)

Columns only in sample: set()
Columns only in big: set()

Columns with different dtypes: {}

Head of sample:


,POLIZA_SUMINISTRO,NUMEROSERIECONTADOR,CONSUMO_REAL,FECHA_HORA,CODI_ANOMALIA,DATA_INICI,DATA_FI
0,JSM5YS4KVQUI5DQA,RMQO6U3MP5TS4QUL,NaN,2024-06-17 00:00:00,163840,2024-07-15,2024-09-13
1,JSM5YS4KVQUI5DQA,RMQO6U3MP5TS4QUL,21.0,2024-06-17 00:56:13,163840,2024-07-15,2024-09-13
2,JSM5YS4KVQUI5DQA,RMQO6U3MP5TS4QUL,7.0,2024-06-17 01:56:13,163840,2024-07-15,2024-09-13



Head of big:


,POLIZA_SUMINISTRO,NUMEROSERIECONTADOR,CONSUMO_REAL,FECHA_HORA,CODI_ANOMALIA,DATA_INICI,DATA_FI
0,JSM5YS4KVQUI5DQA,RMQO6U3MP5TS4QUL,NaN,2024-06-17 00:00:00,163840,2024-07-15,2024-09-13
1,JSM5YS4KVQUI5DQA,RMQO6U3MP5TS4QUL,21.0,2024-06-17 00:56:13,163840,2024-07-15,2024-09-13
2,JSM5YS4KVQUI5DQA,RMQO6U3MP5TS4QUL,7.0,2024-06-17 01:56:13,163840,2024-07-15,2024-09-13


,count_sample,mean_sample,std_sample,min_sample,25%_sample,50%_sample,75%_sample,max_sample,count_big,mean_big,std_big,min_big,25%_big,50%_big,75%_big,max_big
CODI_ANOMALIA,25977.0,115330.695307,63286.753163,32768.0,32768.0,163840.0,163840.0,163840.0,25977.0,115330.695307,63286.753163,32768.0,32768.0,163840.0,163840.0,163840.0
CONSUMO_REAL,21528.0,2.795940,10.903834,0.0,0.0,0.0,0.0,419.0,21528.0,2.795940,10.903834,0.0,0.0,0.0,0.0,419.0



10 IDs overlap out of 10 in sample and 10 in big.



## Step 1 — Load & Light Schema Mapping
We define a minimal **canonical schema** to standardize column names across sources.  
Only map columns that exist; everything else remains untouched.


In [4]:

# === Canonical mapping adapted to your dataset ===
CANONICAL_MAP = {
    # --- time fields ---
    "DATETIME": "datetime",
    "DATE": "date",
    "DATA_INICI": "data_inici",   # start of consumption period
    "DATA_FI": "data_fi",         # end of consumption period
    "YEAR": "year",

    # --- identifiers ---
    "POLIZA_SUMINISTRO": "polissa_id",
    "NUMEROSERIECONTADOR": "num_serie_contador",

    # --- main variable ---
    "CONSUMO_REAL": "consumption",

    # --- anomaly info ---
    "CODI_ANOMALIA": "codi_anomalia",
}


def load_csv_mapped(path):
    """
    Loads the CSV converted from Parquet and applies canonical renaming
    based on Iteration 2 column structure.
    """
    if path is None or not os.path.exists(path):
        raise FileNotFoundError("WORK_CSV not set or file does not exist. Ensure Parquet→CSV ran and a file is available.")

    df = pd.read_csv(path)

    # Normalize column names: strip spaces, uppercase, map through CANONICAL_MAP
    new_cols = []
    for c in df.columns:
        key = c.strip().upper()
        new_cols.append(CANONICAL_MAP.get(key, c))
    df.columns = new_cols

    # Parse datetime and date fields safely
    if "datetime" in df.columns:
        df["datetime"] = pd.to_datetime(df["datetime"], errors="coerce")
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], errors="coerce").dt.date
    if "data_inici" in df.columns:
        df["data_inici"] = pd.to_datetime(df["data_inici"], errors="coerce")
    if "data_fi" in df.columns:
        df["data_fi"] = pd.to_datetime(df["data_fi"], errors="coerce")

    # Derive year if not already present
    if "year" not in df.columns and "date" in df.columns:
        df["year"] = pd.to_datetime(df["date"], errors="coerce").dt.year

    return df


df = load_csv_mapped(WORK_CSV) if WORK_CSV else None
if df is not None:
    print(df.shape)
    display(df.head(3))


(25977, 7)


,polissa_id,num_serie_contador,consumption,FECHA_HORA,codi_anomalia,data_inici,data_fi
0,JSM5YS4KVQUI5DQA,RMQO6U3MP5TS4QUL,NaN,2024-06-17 00:00:00,163840,2024-07-15,2024-09-13
1,JSM5YS4KVQUI5DQA,RMQO6U3MP5TS4QUL,21.0,2024-06-17 00:56:13,163840,2024-07-15,2024-09-13
2,JSM5YS4KVQUI5DQA,RMQO6U3MP5TS4QUL,7.0,2024-06-17 01:56:13,163840,2024-07-15,2024-09-13



## 1.3.8.1 — Data Quality Re‑check
We verify coherence across **periods (years)** and **municipalities**, and produce a short data quality report.
- Missingness per column
- Duplicates
- Basic range sanity for `consumption` if present
- Coverage table by `municipi × year`


In [5]:
# --- Guard: ensure data loaded ---
if df is None:
    raise SystemExit("No data loaded. Make sure a Parquet exists under `data/` and was converted to CSV.")

# --- Missingness summary ---
na_summary = df.isna().mean().reset_index()
na_summary.columns = ["column", "na_ratio"]
_save(na_summary, os.path.join(RESULTS_DIR, "qcheck_missingness.csv"))

# --- Duplicate rows ---
dup_count = df.duplicated().sum()
with open(os.path.join(RESULTS_DIR, "qcheck_duplicates.json"), "w") as f:
    json.dump({"duplicate_rows": int(dup_count)}, f)
print(f"Duplicate rows: {dup_count}")

# --- Consumption sanity (if present) ---
range_report = {}
if "consumption" in df.columns:
    c = df["consumption"]
    range_report = {
        "min": float(c.min(skipna=True)),
        "p01": float(c.quantile(0.01)),
        "median": float(c.median(skipna=True)),
        "p99": float(c.quantile(0.99)),
        "max": float(c.max(skipna=True)),
        "negatives_count": int((c < 0).sum()),
        "zeros_count": int((c == 0).sum()),
    }
with open(os.path.join(RESULTS_DIR, "qcheck_consumption_range.json"), "w") as f:
    json.dump(range_report, f, indent=2)

# --- Coverage by year (simple temporal completeness) ---
if "year" in df.columns:
    cov_year = df.groupby("year").size().reset_index(name="n_records")
    _save(cov_year, os.path.join(RESULTS_DIR, "qcheck_coverage_year.csv"))

# --- (Optional but useful) anomaly code distribution ---
if "codi_anomalia" in df.columns:
    anom = (
        df["codi_anomalia"]
        .value_counts(dropna=False)
        .reset_index()
        .rename(columns={"index": "codi_anomalia", "codi_anomalia": "count"})
    )
    _save(anom, os.path.join(RESULTS_DIR, "qcheck_anomaly_codes.csv"))


[saved] C:\Users\joan\Desktop\FEINA\UPF\Course\Fourth_year\Primer_Trimestre\Project_Management\AB\AB_DataChallenge.Team102D\results\iteration_2\qcheck_missingness.csv
Duplicate rows: 0
[saved] C:\Users\joan\Desktop\FEINA\UPF\Course\Fourth_year\Primer_Trimestre\Project_Management\AB\AB_DataChallenge.Team102D\results\iteration_2\qcheck_anomaly_codes.csv



## 1.3.8.2 — Exploratory Analysis on New/Updated Variables
We detect **candidate new variables** by excluding a small **baseline** set and then:
- Describe numeric columns (`.describe()`)
- List top categories for categorical columns


In [ ]:

baseline_cols = {
    "date","datetime","year",
    "consumption",
    "municipi","districte","barri","codi_postal","seccio_censal",
    "tipus_consum","activitat"
}
# polissa_id	num_serie_contador	consumption	FECHA_HORA	codi_anomalia	data_inici	data_fi
present = set(df.columns)
candidate_new = sorted(list(present - baseline_cols))
pd.DataFrame({"candidate_new_columns": candidate_new}).to_csv(
    os.path.join(RESULTS_DIR, "eda_candidate_new_columns.csv"), index=False
)
print("Candidate new columns:", candidate_new)

# Numeric describe for all numeric columns (including 'new' ones)
num_desc = df.describe(include="number").T
_save(num_desc.reset_index().rename(columns={"index":"column"}),
      os.path.join(RESULTS_DIR, "eda_numeric_describe.csv"))

# Top categories for object category-like columns
cat_cols = [c for c in df.columns if df[c].dtype == "object" and c not in ("datetime","date")]
cat_value_counts = {}
for c in cat_cols:
    vc = df[c].value_counts(dropna=False).head(20)
    cat_value_counts[c] = vc.to_dict()

with open(os.path.join(RESULTS_DIR, "eda_top_categories.json"), "w") as f:
    json.dump(cat_value_counts, f, indent=2)

print(f"[ok] Wrote EDA summaries to {RESULTS_DIR}")


Candidate new columns: ['FECHA_HORA', 'codi_anomalia', 'data_fi', 'data_inici', 'num_serie_contador', 'polissa_id']
[saved] C:\Users\joan\Desktop\FEINA\UPF\Course\Fourth_year\Primer_Trimestre\Project_Management\AB\AB_DataChallenge.Team102D\results\iteration_2\eda_numeric_describe.csv
[ok] Wrote EDA summaries to C:\Users\joan\Desktop\FEINA\UPF\Course\Fourth_year\Primer_Trimestre\Project_Management\AB\AB_DataChallenge.Team102D\results\iteration_2



## 1.3.8.3 — Adjustment of Cleaning Rules if Needed
We implement **minimal, reversible** cleaning. The guiding principle is: *do no harm*.
- Keep raw values; add **flags** for suspicious records.
- Only drop rows if they are clearly invalid (e.g., completely empty or impossible dates).
- Save both **cleaned** data and a small **log** of what changed.


In [7]:

def minimal_cleaning(df):
    df = df.copy()
    changes = {"flags_added": [], "rows_dropped": 0}

    # Flag negative and zero consumptions (do not modify the raw value)
    if "consumption" in df.columns:
        df["flag_negative_consumption"] = df["consumption"] < 0
        df["flag_zero_consumption"] = df["consumption"] == 0
        changes["flags_added"] += ["flag_negative_consumption", "flag_zero_consumption"]

    # Flag known anomaly codes (if present)
    if "codi_anomalia" in df.columns:
        df["flag_anom_32768"] = df["codi_anomalia"] == 32768
        df["flag_anom_163840"] = df["codi_anomalia"] == 163840
        changes["flags_added"] += ["flag_anom_32768", "flag_anom_163840"]

    # Drop absurd dates (keep most rows)
    if "date" in df.columns:
        # consider years 2000..2030 as valid window
        y = pd.to_datetime(df["date"], errors="coerce").dt.year
        mask_valid_year = y.between(2000, 2030)
        before = len(df)
        df = df[mask_valid_year].copy()
        changes["rows_dropped"] += before - len(df)

    # Deduplicate strictly identical rows
    before = len(df)
    df = df.drop_duplicates()
    changes["rows_dropped"] += before - len(df)

    return df, changes

clean_df, changes = minimal_cleaning(df)
with open(os.path.join(RESULTS_DIR, "cleaning_changes.json"), "w") as f:
    json.dump(changes, f, indent=2)
print("Cleaning changes:", json.dumps(changes, indent=2))

# Persist cleaned outputs
clean_csv = os.path.join(RESULTS_DIR, "cleaned_dataset_v2.csv")
clean_parquet = os.path.join(RESULTS_DIR, "cleaned_dataset_v2.parquet")
clean_df.to_csv(clean_csv, index=False)
try:
    clean_df.to_parquet(clean_parquet, index=False)
except Exception as e:
    print(f"[warn] Could not write parquet: {e}")
print(f"[ok] Saved cleaned data to {RESULTS_DIR}")


Cleaning changes: {
  "flags_added": [
    "flag_negative_consumption",
    "flag_zero_consumption",
    "flag_anom_32768",
    "flag_anom_163840"
  ],
  "rows_dropped": 0
}
[ok] Saved cleaned data to C:\Users\joan\Desktop\FEINA\UPF\Course\Fourth_year\Primer_Trimestre\Project_Management\AB\AB_DataChallenge.Team102D\results\iteration_2



## Summary — What to Do Next
- Review the CSVs in `results/iteration_2/`:
  - `qcheck_missingness.csv` and `qcheck_coverage_municipi_year.csv`
  - `eda_candidate_new_columns.csv`, `eda_numeric_describe.csv`, `eda_top_categories.json`
  - `cleaned_dataset_v2.csv` and `cleaning_changes.json`
- If you find issues that require **adjusting rules** (e.g., clamp negatives, impute missing), 
  add them to `minimal_cleaning()` and **document the rationale** inline.
- This notebook finishes **Task 1.3.8** and prepares inputs for **1.3.9 Feature Engineering**.
